In [ ]:
#install.packages("Rpdb")
#install.packages("bio3d")

In [ ]:
#install.packages("remotes")
#remotes::install_github("nvelden/NGLVieweR")

In [ ]:
library(httr)
library(xml2)

# Acceso a Pfam

- **Base de datos Pfam:**
  - Es una gran colección de familias de dominios de proteínas.
- **Cada familia Pfam:**
  - Representada por múltiples alineamientos de secuencias
    y un modelo oculto de Markov (HMM).
- **Entrada Pfam-A:**
  - Generalmente conocida como una entrada Pfam-A.
- **Contenido de cada entrada Pfam-A:**
  - Alineamiento de semillas curado que contiene un pequeño conjunto de miembros
    representativos de la familia.
  - Perfil HMM construido a partir del alineamiento de semillas.
  - Alineamiento completo generado automáticamente, que contiene todas las
    secuencias de proteínas detectables pertenecientes a la familia.
- **Generación del alineamiento completo:**
  - Realizado mediante búsquedas de HMM de perfil en bases de datos de
    secuencias primarias.
- **Se accede por la web de interpro.**
  - Previamente tenía una web propia.
- **URL**
  - Web: https://www.ebi.ac.uk/interpro/entry/pfam/#table
  - API: "https://www.ebi.ac.uk/interpro/api/"
  - API reference: https://pfam-docs.readthedocs.io/en/latest/api.html

## Recuperar datos de las familias disponibles

Uno de los servicios permite recuperar el nombre, código de acceso y descripción de todas las familias de proteínas disponibles.

In [ ]:
url <- "https://www.ebi.ac.uk/interpro/api/entry/integrated/pfam"
respuesta <- GET(
  url,
  query = list(
    type = "family",
    format = "json",
    page_size = 10 # 11320
  ),
)

In [ ]:
results <- content(respuesta, as = "parsed", encoding = "utf-8")
# results
names(results)

La respuesta tiene 4 campos:

- count: es el número total de resultados
- next: link a la siguiente página de resultados
- previous: links a la página de resultados anteriores
- results: tiene la lista de resultados:
  - cada resultado tiene un campo llamado metadata.
  - la metadata tiene:
    - el accession
    - el nombre
    - la base de datos de origen
    - el tipo de entrada
    - el identificador de Interpro

In [ ]:
result_df <- as.data.frame(
  t(
    sapply(
      results$results,
      function(x) return(unlist(x$metadata))
    )
  )
)
result_df

Podemos buscar por:

- repeat: motivo repetitivos
- domain: dominos de proteínas.
- coiled coil: motivos coiled-coil.
- disordered: regiones desordenadas.
- conserved site: sitios conservados.

## Recuperar anotaciones

In [ ]:
family_acc <- "PF00082"

url <- paste(
  "https://www.ebi.ac.uk/interpro/api/entry/pfam",
  family_acc,
  sep = "/"
)
respuesta <- GET(
  url,
  query = list(
    format = "json"
  ),
)

In [ ]:
content(respuesta)$metadata$name
content(respuesta)$metadata$description[[1]]$text
content(respuesta)$metadata$wikipedia$extract
content(respuesta)$metadata


## Buscar proteínas que contienen un dominio pfam

In [ ]:
pfam_domain <- "PF00082"

url <- paste(
  "https://www.ebi.ac.uk/interpro/api/protein/Uniprot/entry/pfam",
  pfam_domain,
  sep = "/"
)
url

In [ ]:
response <- GET(
  url,
  query = list(format = "json", page_size = 100),
)

In [ ]:
t(
  sapply(
    content(response, as = "parsed")$results,
    function(x) unlist(x$metadata)
  )
)

## Buscar dominios pfam en una proteína



In [ ]:

uniprot_acc <- "P0AA25" # Thioredoxin 1 de E. coli K 12

url <- paste(
  "https://www.ebi.ac.uk/interpro/api/entry/pfam/protein/Uniprot",
  uniprot_acc,
  sep = "/"
)
url

In [ ]:
response <- GET(
  url,
  query = list(format = "json", page_size = 100),
)

In [ ]:
t(
  sapply(
    content(response, as = "parsed")$results,
    function(x) unlist(x$metadata)
  )
)

## Buscar pfam domains en una secuencia

In [ ]:
## Buscar dominios en un secuencia de una proteína
url <- "https://www.ebi.ac.uk/Tools/services/rest/iprscan5/run"
seq <- "MLPGLALLLLAAWTARALEVPTDGNAGLLAEPQIAMFCGRLNMHMNVQNGKWDSDPSGTKTCIDTKEGILQYCQEVYPELQITNVVEANQPVTIQNWCKRGRKQCKTHPHFVIPYRCLVGEFVSDALLVPDKCKFLHQERMDVCETHLHWHTVAKETCSEKSTNLHDYGMLLPCGIDKFRGVEFVCCPLAEESDNVDSADAEEDDSDVWWGGADTDYADGSEDKVVEVAEEEEVAEVEEEEADDDEDDEDGDEVEEEAEEPYEEATERTTSIATTTTTTTESVEEVVREVCSEQAETGPCRAMISRWYFDVTEGKCAPFFYGGCGGNRNNFDTEEYCMAVCGSAMSQSLLKTTQEPLARDPVKLPTTAASTPDAVDKYLETPGDENEHAHFQKAKERLEAKHRERMSQVMREWEEAERQAKNLPKADKKAVIQHFQEKVESLEQEAANERQQLVETHMARVEAMLNDRRRLALENYITALQAVPPRPRHVFNMLKKYVRAEQKDRQHTLKHFEHVRMVDPKKAAQIRSQVMTHLRVIYERMNQSLSLLYNVPAVAEEIQDEVDELLQKEQNYSDDVLANMISEPRISYGNDALMPSLTETKTTVELLPVNGEFSLDDLQPWHSFGADSVPANTENEVEPVDARPAADRGLTTRPGSGLTNIKTEEISEVKMDAEFRHDSGYEVHHQKLVFFAEDVGSNKGAIIGLMVGGVVIATVIVITLVMLKKKQYTSIHHGVVEVDAAVTPEERHLSKMQQNGYENPTYKFFEQMQN"

body <- list(
  sequence = seq,
  email = "my_email@gmail.com",
  appl = "PfamA",
  goterms = "false",
  pathways = "false"
)

response <- POST(
  url,
  body = body
)

In [ ]:
job_id <- content(response)

In [ ]:
## Tengo que verificar el estado del trabajo
url <- paste(
  "https://www.ebi.ac.uk/Tools/services/rest/iprscan5/status",
  job_id,
  sep = "/"
)

response <- GET(url, query = query)

In [ ]:
content(response)

In [ ]:
## Recupero el resultado
url <- paste(
  "https://www.ebi.ac.uk/Tools/services/rest/iprscan5/result",
  job_id,
  "json",
  sep = "/"
)

response <- GET(url, query = query)

In [ ]:
results <- content(response)

In [ ]:
t(
  sapply(
    results$results[[1]]$matches,
    function(x) {
      c(
        acceesion = x$signature$accession,
        name = x$signature$name,
        description = x$signature$description,
        type = x$signature$entry$type,
        start = x$locations[[1]]$start,
        end = x$locations[[1]]$end
      )
    }
  )
)


## Descargar alineamientos

Podemos descargas varios tipos de alineamientos de pfam


In [ ]:
pfam_accession <- "PF00085"

url <- paste(
  "https://www.ebi.ac.uk/interpro/wwwapi//entry/pfam",
  pfam_accession,
  sep = "/"
)

outfile <- "PF00085.sto.gz"

GET(
  url,
  query = list(
    annotation = "alignment:full",
    download = TRUE
  ),
  write_disk(outfile, overwrite = TRUE)
)


In [ ]:
library(R.utils)
gunzip(outfile)


In [ ]:
# Descargamos alineamientos Seed
accession <- "PF00085"
url <- paste0("https://pfam.xfam.org/family/", accession, "/alignment/seed")
seed_msa <- GET(url)

In [ ]:
# Lo guardamos en el disco
write(content(seed_msa, as="text"), file="pf_00085_seed.stockholm")

In [ ]:
# Descargamos alineamientos full

accession <- "PF00085"
url <- paste0("https://pfam.xfam.org/family/", accession, "/alignment/full")
full_msa <- GET(url)

In [ ]:
# Lo guardamos en el disco
write(content(full_msa, as="text"), file="pf_00085_full.stockholm")

Podemos descargar  proteomas representativos con diferente grado de indentidad de secuencia,
que pertenecen a una familia.

- RP15 Proteinas con 15% de identidad o mayor
- RP35 Proteinas con 35% de identidad o mayor
- RP55 Proteinas con 55% de identidad o mayor
- RP75 Proteinas con 75% de identidad o mayor

Para ello, hay que modificar la url.

In [ ]:
url <- paste0("https://pfam.xfam.org/family/", accession, "/alignment/rp15")
url <- paste0("https://pfam.xfam.org/family/", accession, "/alignment/rp35")
url <- paste0("https://pfam.xfam.org/family/", accession, "/alignment/rp55")
url <- paste0("https://pfam.xfam.org/family/", accession, "/alignment/rp75")

# Acceso a RSCB PDB

Al igual que NCBI y Uniprot, PDB permite realizar búsqueda de texto.
Aunque estas son bastante más complicadas de realizar que en los servicios anteriores.

La consulta se tiene que escribir en formato **json**.

Vamos a provar con una búsquea simple.

La respuesta también está en formato json.

In [ ]:
library(httr)

In [ ]:
url <- "https://search.rcsb.org/rcsbsearch/v2/query"

query <- '{
  "request_options": {
    "return_all_hits": true
  },
  "query": {
    "type": "terminal",
    "service": "full_text",
    "parameters": {
      "value": "phosphofructokinase"
    }
  },
  "return_type": "entry"
}'

respuesta <- GET(url, query = list(json = query))

In [ ]:
pdb_search <- content(respuesta)

In [ ]:
sapply(pdb_search$result_set, unlist)

In [ ]:
# Recuperamos la cantidad de hits.
pdb_search$total_count

In [ ]:
# Recuperamos todos los resultados
length(pdb_search$result_set)

In [ ]:
pdb_ids <- sapply(pdb_search$result_set, function(x) x$identifier)

In [2]:

library(bio3d)
library(Rpdb)

Warning message:
"package 'bio3d' was built under R version 4.1.3"
Loading required package: rgl


Attaching package: 'Rpdb'


The following objects are masked from 'package:bio3d':

    is.pdb, read.pdb, write.pdb


The following objects are masked from 'package:base':

    norm, replicate, unsplit




In [ ]:
# Podemos descargar el archivo PDB

bio3d::get.pdb(pdb_ids[1])

In [3]:
# Podemos leer los datos del PDB
pdb1 <- bio3d::read.pdb('./1MTO.pdb')

In [5]:
pdb1$atom

type,eleno,elety,alt,resid,chain,resno,insert,x,y,z,o,b,segid,elesy,charge
<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>
ATOM,1,N,NA,MET,A,1,NA,-44.169,-36.847,78.429,1,58.78,NA,N,NA
ATOM,2,CA,NA,MET,A,1,NA,-44.764,-37.009,79.792,1,58.16,NA,C,NA
ATOM,3,C,NA,MET,A,1,NA,-46.223,-36.567,79.801,1,57.26,NA,C,NA
ATOM,4,O,NA,MET,A,1,NA,-46.827,-36.345,78.748,1,57.39,NA,O,NA
ATOM,5,CB,NA,MET,A,1,NA,-43.997,-36.171,80.822,1,58.69,NA,C,NA
ATOM,6,CG,NA,MET,A,1,NA,-42.496,-36.398,80.845,1,59.93,NA,C,NA
ATOM,7,SD,NA,MET,A,1,NA,-41.672,-35.409,82.117,1,60.99,NA,S,NA
ATOM,8,CE,NA,MET,A,1,NA,-41.036,-36.721,83.228,1,60.05,NA,C,NA
ATOM,9,N,NA,LYS,A,2,NA,-46.779,-36.438,81.002,1,55.56,NA,N,NA


In [ ]:
library(NGLVieweR)

In [ ]:
NGLVieweR("./1MTO.pdb") %>% addRepresentation("cartoon")